In [157]:
# !pip install gym_anytrading
# !pip install stable-baselines3==1.7.0
# !pip install stable-baselines gym-anytrading gym
# !pip install pygame --pre

In [158]:
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gymnasium as gym
from gymnasium import spaces


from typing import Any


import torch
Tensor = torch.Tensor
import torch.nn as nn


In [159]:
path_m5 : str = r"C:\Users\Nemo\Desktop\sth-financial\XAUUSDM5.csv"
path_m15: str = r"C:\Users\Nemo\Desktop\sth-financial\XAUUSDM15.csv"

In [160]:
df = pd.read_csv(path_m15 , 
				   encoding = 'utf-16', 
				   names =  ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', '0'] )

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', drop=True, inplace = True)
df.drop('0', axis=1)


# variables

Open_ = df['Open']
close = df['Close']
# volume = data['Volume']
high = df['High']
time = df.index

Round = 4

rsi_len = 14

std_len = mean_len = 5

Vol_ma_1 = ma_1 = 10

Vol_ma_2 = ma_2 = 20


# X = df[[ 'change', 'rsi',   'macd' ]]
# Y = df[[ 'buy', 'sell']]

# X_train, X_test, y_train, y_test = \
# train_test_split( X,
#                  Y, 
#                  test_size= 0.30
#                  , shuffle = True)

In [161]:
df.head()

,Open,High,Low,Close,Volume,0
Date,,,,,,
2025-04-16 01:00:00,3230.96,3233.35,3229.84,3232.16,482,0
2025-04-16 01:15:00,3232.15,3235.25,3231.47,3235.25,368,0
2025-04-16 01:30:00,3234.82,3238.90,3234.37,3238.82,733,0
2025-04-16 01:45:00,3238.76,3242.23,3237.48,3241.47,752,0
2025-04-16 02:00:00,3241.52,3241.74,3238.36,3241.73,756,0


#3

In [162]:

def calculate_RSI(data, time_window = 14):
		delta = data.diff()

		up, down = delta.copy(), delta.copy()
		up[up < 0] = 0
		down[down > 0] = 0

		roll_up = up.rolling(time_window).mean()
		roll_down = down.abs().rolling(time_window).mean()

		RS = roll_up / roll_down
		RSI = 100.0 - (100.0 / (1.0 + RS))

		return round(RSI, 3)

def norming(row, on: str ):
	if max(row['buy'], row['sell'], row['hold']) == row[on]:
		return 1
	else:
		return 0

def ABS(row ):
	if row['sell']:
		return row["sell"].abs()
	else:
		return row['sell']

# def moving_avg(data, window_size):
#     if isinstance(data, np.ndarray):
#         return round(np.convolve(data, np.ones(window_size)/window_size, mode='valid'), 5)
#     if isinstance(data, pd.Series):
#         return round(data.rolling(window=window_size, min_periods=1).mean(), 5)

# def macd(data, fast, slow):
#     if isinstance(data, np.ndarray):
#         ma1 =  np.convolve(data, np.ones(fast)/fast, mode='valid')
#         ma2 =  np.convolve(data, np.ones(slow)/slow, mode='valid')
#         return ma2 - ma1
#     if isinstance(data, pd.Series):
#         ma1 =  data.rolling(fast, min_periods=1).mean()
#         ma2 =  data.rolling(slow, min_periods=1).mean()
#         return ma2 - ma1

# def std(series : pd.Series , window: int):
#     rolling_std = series.rolling(window=window).std()
#     return  rolling_std

def clc_mean(series : pd.Series , window: int) :
		rolling_mean = series.rolling(window=window).mean()
		return  rolling_mean

# def moving_avg_std(series, window):
#     """Calculate the moving average and standard deviation of a pandas series, and then divide the two"""
#     rolling_mean = series.rolling(window=window).mean()
#     rolling_std = series.rolling(window=window).std()
#     return rolling_mean - rolling_std
				
# def lr_schedule_1(epoch, lr):
#     if epoch < 3:
#         return lr
#     else:
#         return lr * tf.math.exp(-0.1)

# def lr_schedule_2(epoch, lr, decay_rate=0.5, decay_steps = 3 ):
#     return lr * decay_rate**(np.floor(epoch / decay_steps))

In [163]:
def creat_data(df: pd.DataFrame):
	df['difference'] = diff = round( high - Open_ , Round )
	df['cadle-type'] = cndl = round( close - Open_ , Round )
	df['hike'] = diff / cndl
	# df['change'] = change = round( Open_ - moving_avg(Open_, ma_1 ) , Round )

	df['percent'] = percent = round( (close - Open_) / close , Round ) * 100

	df['rsi'] = rsi = round(calculate_RSI( Open_ , rsi_len ) / 100,  Round )

	# df['std'] = Std = round(std(Open_ , std_len), Round )

	df['mean'] = Mean = round(clc_mean(Open_ , std_len), Round )

	df['buy'] = buyers = np.where( (Open_ > clc_mean(Open_,mean_len).shift(1)) & (cndl >= .65), 1 ,0 )

	df['sell'] = buyers = np.where( (Open_ < clc_mean(Open_,mean_len).shift(1)) & (abs(cndl) >= .65)  , 1 ,0 )

	df['hold'] = no_action = np.where(df['buy'] == df['sell'], 1, 0)

	# df['action'] = np.where(buyers == 1, 1,0)

	df = df.dropna(axis=0)

	return df

In [164]:
df = creat_data(df)

# df.tail()

data = df[['difference', 'cadle-type' ]]
data.head()

,difference,cadle-type
Date,,
2025-04-16 04:30:00,1.77,0.78
2025-04-16 04:45:00,5.77,4.53
2025-04-16 05:00:00,4.71,0.67
2025-04-16 05:15:00,1.71,0.34
2025-04-16 05:30:00,2.49,-2.30


In [ ]:
from pandas import DataFrame

window_size = 10
num_features = df.shape[1]  # = 3 (Close, Volume, MA10)


class MarketEnv(gym.Env):
	def __init__(self, df: pd.DataFrame, win_size: int):
		super(MarketEnv, self).__init__()
		self.df: DataFrame = df.reset_index(drop=True).astype(np.float32)
		self.win_size: int = win_size
		self.num_features: int = df.shape[1]
		self.current_step: int = win_size
		self.reward: float = 0
		self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)
		self.observation_space = spaces.Box(
			low=-np.inf,
			high=np.inf,
			shape=(win_size, self.num_features),
			dtype=np.float32
		)

		self.max_steps = len(df) - win_size - 1
		self.step_count = 0
		self.state = None

	def reset(self, *, seed=None, options=None):
		super().reset(seed=seed) 
		self.current_step = self.win_size
		self.step_count = 0
		self.state = self._get_observation()
		return self.state, {}

	def _get_observation(self):
		window: DataFrame = self.df.iloc[self.current_step - self.win_size : self.current_step]
		return window.to_numpy()

	def step(self, action):
		action = np.clip(action, -1, 1)

		# نمونه پاداش ساده: سعی کن مقدار اولین ویژگی را کم یا زیاد کنی تا به مقدار خاصی برسی
		feature_val = self.df.iloc[self.current_step, 0]
		price_now = self.df.iloc[self.current_step]['Close']
		price_prev = self.df.iloc[self.current_step - 1]['Close']
		position = action[0]

		self.reward = (price_now - price_prev) * np.abs(position)  # simulate PnL

		self.current_step += 1
		self.step_count += 1
		
		terminated = False
		truncated: bool = self.step_count >= self.max_steps
		self.state = self._get_observation()
		info = {}
		return self.state, self.reward, terminated, truncated, info

	def render(self, mode="human") -> None:
		print(f"Step: {self.step_count} | reward:{self.reward :.4f}")



In [166]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Actor(nn.Module):
	def __init__(self, input_dim, action_dim):
		super().__init__()

		self.fc = nn.Sequential(
			nn.Linear(input_dim, 128), # fc1
			nn.ReLU(),
			nn.Linear(128, 128), # fc2
			nn.ReLU(),
			nn.Linear(128, action_dim), # Mu   ,
			nn.Tanh()         
		)

		self.log_std = nn.Parameter(torch.zeros(action_dim))

	def forward(self, x):
		mu = self.fc(x)
		log_std = torch.clamp(self.log_std, min=-20, max=2)
		std = log_std.exp()
		return mu, std

class Critic(nn.Module):
	def __init__(self, input_dim):
		super().__init__()
		self.fc : nn.Sequential = nn.Sequential(
				nn.Linear(input_dim, 128) , # 1
				nn.ReLU(),
				nn.Linear(128, 128), # 2
				nn.ReLU(),
				nn.Linear(128, 1), # value

		)

	def forward(self, x: Tensor) -> Tensor:
		value: Tensor = self.fc(x)
		return value

# تبدیل observation به بردار 1D برای ورودی شبکه
def preprocess_obs(obs : np.ndarray | Tensor) -> torch.Tensor:
	return torch.tensor(obs.flatten(), dtype=torch.float32)


In [167]:
assert not data.isnull().any().any(), "NaN exists in dataframe!"

data.describe()

,difference,cadle-type
count,5395.000000,5395.000000
mean,2.832489,-0.001833
std,2.831925,4.261245
min,0.000000,-37.380000
25%,0.910000,-1.995000
50%,2.050000,0.050000
75%,3.820000,2.090000
max,35.140000,32.830000


In [ ]:
window_size = 5


env = MarketEnv(df=data, win_size=window_size)
obs, _ = env.reset()

actor = Actor(input_dim=window_size * env.num_features, action_dim=1)
critic = Critic(input_dim=window_size * env.num_features)

optimizerA = optim.Adam(actor.parameters(), lr=1e-3)
optimizerC = optim.Adam(critic.parameters(), lr=1e-3)

gamma = 0.99

for episode in range(100):
    obs, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        state = preprocess_obs(obs)
        # print(f"state: {state.reshape(-1, 3)}")
        # print(f"state_isnan: {torch.isnan(state).any()}\nstate_isinf: {torch.isinf(state).any()}" )

        mu, std = actor(state)
        # print(f"mu: {mu}, std: {std}")
        dist = torch.distributions.Normal(mu, std)
        action = dist.sample()
        action_clipped = action.clamp(-1, 1).detach().numpy()

        next_obs, reward, terminated, truncated, info = env.step(action_clipped)
        done = terminated or truncated

        total_reward += reward

        next_state = preprocess_obs(next_obs)

        # محاسبه value فعلی و next value
        value = critic(state)
        next_value = critic(next_state).detach()

        # هدف Critic (target)
        target = reward + gamma * next_value * (1 - int(done))

        # محاسبه خطا Critic
        critic_loss = F.mse_loss(value, target)

        # محاسبه advantage برای Actor
        advantage = (target - value).detach()

        log_prob = dist.log_prob(action)
        actor_loss = -(log_prob * advantage)

        # به‌روزرسانی Critic
        optimizerC.zero_grad()
        critic_loss.backward()
        optimizerC.step()

        # به‌روزرسانی Actor
        optimizerA.zero_grad()
        actor_loss.backward()
        optimizerA.step()

        obs = next_obs

    print(f"Episode {episode} | Total Reward: {total_reward}")


Episode 0 | Total Reward: -71230.671875
Episode 1 | Total Reward: -70698.015625
Episode 2 | Total Reward: -69738.53125
Episode 3 | Total Reward: -69734.234375


In [ ]:
from gymnasium.envs.registration import register

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

register(
	id='MarketEnv-v0',
	entry_point=MarketEnv, # type: ignore
)


env = DummyVecEnv([
	lambda: Monitor(
		gym.make('MarketEnv-v0', df=df, win_size=window_size)
	)
])


In [ ]:
from stable_baselines3.common.callbacks import EvalCallback

eval_env = DummyVecEnv([
	lambda: gym.make('MarketEnv-v0', df=df, win_size=window_size)
])
eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
							 log_path='./logs/', eval_freq=500,
							 deterministic=True, render=False)




In [ ]:
from stable_baselines3 import DDPG

# ایجاد محیط
# env  = gym.make('MarketEnv-v0', df=df, win_size=window_size)

# ساخت مدل
model = DDPG(
	policy="MlpPolicy",
	env=env,
	verbose=1,
	learning_rate=1e-3,
	buffer_size=100000,
	learning_starts=1000,
	batch_size=64,
	tau=0.005,
	gamma=0.99,
	device='cuda' if torch.cuda.is_available() else 'cpu'
)

# آموزش مدل
model.learn(total_timesteps=2, callback=eval_callback)

# ذخیره مدل
# model.save("ddpg_market_env")

# لود مدل (بعداً)
# model = DDPG.load("ddpg_market_env", env=env)


In [ ]:
# ارزیابی مدل


obs= env.reset() # type: ignore

actions = []
for _ in range(100):
	action, _ = model.predict(obs, deterministic=True) # type: ignore
	obs, reward, done, rec = env.step(action) # type: ignore
	# env.render()
	actions.append(action)
	if done:
		obs = env.reset()

In [ ]:
rewards = []
for _ in range(100):
	action, _ = model.predict(obs, deterministic=True)
	obs, reward, done, _ = env.step(action)
	actions.append(action)
	rewards.append(reward)
	if done:
		obs = env.reset()

plt.plot(rewards)
plt.title("Rewards Over Time")
plt.grid()
plt.show()


In [ ]:
model.predict(obs)